In [ ]:
import os
import numpy as np
from imutils import paths
from keras.models import Model
from keras.layers import AveragePooling2D,Dense,Dropout,Flatten
from keras.applications import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img,img_to_array
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer

base_dir=r"C:\Users\pc\Downloads\archive (2)\data"
categories=["with_mask","without_mask"]
data=[]
labels=[]

for cat in categories:
    path=os.path.join(base_dir,cat)
    for img in os.listdir(path):
        img_path=os.path.join(path,img)
        images=load_img(img_path,target_size=(224,224))
        images=img_to_array(images)
        images=preprocess_input(images)
        data.append(images)
        labels.append(cat)

data=np.array(data,dtype="float32")
labels=np.array(labels)

lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

X_train,X_test,y_train,y_test=train_test_split(data,labels,stratify=labels,test_size=0.2,random_state=42)

baseModel=MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet')
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name="flatten")(headModel)
headModel=Dense(128,activation="relu")(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation="softmax")(headModel)
model=Model(inputs=baseModel.input,outputs=headModel)

for layer in baseModel.layers:
    layer.trainable=False
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20)

model.save("mask_detection.h5")